In [1]:
!pip install gmaps

In [11]:
# Dependencies and Setup
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import gmaps
import json
# Import API key
from config import g_key


In [12]:
file = pd.read_csv("../WeatherPy/weather_data.csv")
file.dropna()
file.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,0,carnarvon,70,AU,1619818645,55,-24.8667,113.6333,18.00,4.63
1,1,namibe,16,AO,1619818806,74,-15.1961,12.1522,24.32,0.75
2,2,ilebo,100,CD,1619818808,98,-4.3167,20.5833,22.28,1.57
3,3,leshukonskoye,100,RU,1619818809,78,64.8989,45.7622,0.76,1.98
4,4,skjern,99,DK,1619818810,74,55.9500,8.5000,6.67,3.00


In [13]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
temp_file=file.loc[(file['Max Temperature']>=20)&(file['Max Temperature']<=30)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
13,13,guerrero negro,0,MX,1619818822,25,27.9769,-114.0611,29.33,6.87
33,33,pisco,0,PE,1619818764,73,-13.7000,-76.2167,22.00,7.20
40,40,tsaratanana,0,MG,1619818855,83,-16.7833,47.6500,22.12,3.78
58,58,cayenne,0,GF,1619818878,83,4.9333,-52.3333,26.11,2.06
98,98,cidreira,0,BR,1619818929,72,-30.1811,-50.2056,20.43,6.80


In [18]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()

    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Name
0,13,guerrero negro,0,MX,1619818822,25,27.9769,-114.0611,29.33,6.87,Hotel Los Caracoles
1,33,pisco,0,PE,1619818764,73,-13.7000,-76.2167,22.00,7.20,"Hotel Paracas, a Luxury Collection Resort, Par..."
2,40,tsaratanana,0,MG,1619818855,83,-16.7833,47.6500,22.12,3.78,
3,58,cayenne,0,GF,1619818878,83,4.9333,-52.3333,26.11,2.06,Mercure Cayenne Royal Amazonia
4,98,cidreira,0,BR,1619818929,72,-30.1811,-50.2056,20.43,6.80,Hotel Araçá
5,125,sabha,0,LY,1619818963,7,27.0377,14.4283,30.00,5.13,Mountain Hotel
6,140,gunjur,0,GM,1619818983,73,13.2019,-16.7339,23.00,3.60,Coco Ocean Resort & Spa
7,162,gasa,0,PS,1619819011,56,31.5000,34.4667,21.11,1.54,West All Suite Hotel Ashdod - מלון ווסט אשדוד
8,171,urumqi,0,CN,1619818925,15,43.8010,87.6005,24.00,7.00,Hilton Urumqi
9,188,dezful,0,IR,1619819044,14,32.3811,48.4058,27.96,1.11,Tabib Traditional Hotel


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]  
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))